In [90]:
import numpy as np
import random
from sage.rings.finite_rings.finite_field_constructor import FiniteField
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.coding.goppa_code import GoppaCode
import math
from itertools import combinations


In [91]:
from sympy import Matrix, Rational, mod_inverse, pprint,  shape

In [94]:
class Attack():
    def __init__(self,m , t, p, l):
        self.n = int( 2**(m) )
        self.m = m
        self.t = t
        self.p = p
        self.l = l
        self.k = int( self.n - self.m * self.t)
        self.modulus = 2
        self.iteration_dict = { "McEliece_iteration":0 ,"LeeBrickell_iteration": 0, "Leon_iteration":0     }
    def is_irreducible(self, poly, field):
        # Check if the polynomial is irreducible over the given field
        return poly.is_irreducible()
    
    def generate_random_goppa_code(self):
        F = FiniteField(2^self.m)
        R.<x>= PolynomialRing(F)
        dim = 0
        iter_err = 0
        while dim != self.n - self.m*self.t:
            while iter_err < 2^10:
                iter_err += 1
                g = R.irreducible_element(self.t)
                
                if g.is_irreducible():
                    L = [a for a in F.list() if g(a)!= 0]
                    #print("dim:", len(L), "\n")
                    #print([a for a in F.list() if g(a)!= 0])
                    np.random.shuffle(L)
                    C = codes.GoppaCode(g,L[:self.n])
                    dim = C.dimension()
                    if dim == self.n - self.m*self.t:
                        print(iter_err, "trials untils  generated")
                        return C.parity_check_matrix(), C.generator_matrix()
        return False


    def is_echelon_form(self,matrix_rref):
        diagonal = list()
        for i in range(self.k):
            diagonal.append( matrix_rref[i,i] )
        if sum(diagonal) == self.k:
            return True
        else:
            return False
        
    def Transform_echelon_Matrix_mod2(self, x):
        numer, denom = x.as_numer_denom()
        return numer*mod_inverse(denom,self.modulus) % self.modulus

    def echelon_form(self,matrix):
        #Find row-reduced echolon form of matrix modulo 2:
        matrix_rref = matrix.rref( iszerofunc = lambda x: x % self.modulus == 0)[0].applyfunc(lambda x:  self.Transform_echelon_Matrix_mod2(x)   )
        if self.is_echelon_form(matrix_rref):
            return matrix_rref, True
        else:
            return matrix_rref,False

    def Generate_S_invertible(self):
        condition = True
        while condition:
            # Ensuring the input for permutation is clearly an integer.
            permutation_input = int(self.k * self.k)
            S = np.random.permutation(permutation_input).reshape(self.k, self.k)
            S = np.mod(S, self.modulus)  # Applying modulo operation to ensure elements are 0 or 1.

            # Checking if the determinant is non-zero for invertibility
            # Convert S to a float type matrix for determinant calculation.
            if np.linalg.det(S.astype(float)) != 0:
                condition = False
        return S
    def generate_permutation_matrix(self):
        identity_matrix = np.eye(int(self.n), dtype = int)
        permutation = np.random.permutation(self.n)
        permutation_matrix = identity_matrix[permutation]
        return permutation_matrix
        
    def generate_public_key(self):
        parity_matrix, generator_matrix = self.generate_random_goppa_code()
        self.parity_matrix = np.array( parity_matrix).astype(int) 
        self.generator_matrix = np.array(generator_matrix).astype(int)
        # verify if  se G.H^T = 0
        validate = np.mod(np.dot(self.generator_matrix,np.transpose(self.parity_matrix)), self.modulus )
        if (validate == np.zeros( (self.k, self.n - self.k ), dtype= int) ).all():
            #print("True")
            pass
        else:
            pass
            #print("False")


        control = True
        iteration = 0
        while control:
                S = self.Generate_S_invertible() 
                P = self.generate_permutation_matrix()
                Public_key = np.mod(np.dot( np.mod( np.dot(S,self.generator_matrix )  , 2)  , P), 2).astype(int)
                sympy_matrix = Matrix(Public_key)
                # verify if public key is full rank 
                echelon_matrix, control =  self.echelon_form(sympy_matrix)
                if control == True:
                    control = False
                    return Public_key
                else:
                    control = True
        
    def GaussianEliminationMod2(self, matrix):
        Matrix_A = matrix.copy()
        # Reduce each of the first k columns 
        for j in range(self.k):       
            # look for a pivot
            pivot_i = -1
            for i in range(j,self.k):
                if Matrix_A[i,j] == 1:
                    pivot_i = i
                    break
            #if pivot_i == -1:
            #    raise ValueError("first k columns do not have full rank") # is no longer to check this 
            
            assert(Matrix_A[pivot_i,j] == 1)

            # swap rows j and pivot_i 
            if pivot_i != j:
                tmp = Matrix_A[pivot_i,:].copy()
                Matrix_A[pivot_i,:] = Matrix_A[j,:].copy()
                Matrix_A[j,:] = tmp
            
            # zero out column j in other rows
            for i in range(self.k):
                if i != j and Matrix_A[i,j]==1:
                    Matrix_A[i,:] = np.mod( Matrix_A[i,:] + Matrix_A[j,:], self.modulus )
        return Matrix_A
                    
        
    def verificar_inversa_public_key(self,indices_aleatorios):
        # Converte a matriz para o tipo de dados 'int'
        matrix_I = np.array( self.Public_key )[:, indices_aleatorios].astype(int)
        if np.linalg.det( matrix_I ) != 0:
            return True
        else:
            return False
    def NextConfiguration(self, z, sigma, Algorithm):
        permutation_gives_fullrank_block = False
        while not permutation_gives_fullrank_block:
            if Algorithm    ==  "McEliece_information":
                self.iteration_dict["McEliece_iteration"] += 1
            
            elif Algorithm == "LeeBrickell":
                self.iteration_dict["LeeBrickell_iteration"] += 1
            elif Algorithm == "Leon":
                self.iteration_dict["Leon_iteration"] += 1
            else:
                print("Error")
                
            theta = random.sample([i for i in range(self.n)], self.n)
            try:
                G = self.GaussianEliminationMod2(self.Public_key[:, theta])        
            except:
                pass
            else:
                permutation_gives_fullrank_block = True
        # update z
        zz = z[theta]
        z = np.mod(zz + zz[:self.k].dot( G ) , self.modulus)   # array
        # update sigma
        sigma = sigma[theta]
        return G, z, sigma, theta[:self.k]
    def DecodeError_McEliece_information(self):
        assert(len(self.ciphertext) == self.n)
        z = self.ciphertext.copy()
        sigma = np.arange(self.n)
        
        while True:
            G, z, sigma, information_set_decoding = self.NextConfiguration(z, sigma,"McEliece_information" )
            # check for success
            assert(all(z[:self.k]==0))
            if sum(z[self.k:]) == self.t:
                break
        # un-permute z
        e = np.zeros(self.n, dtype=np.int32)
        for i, loc in enumerate(sigma):
            e[loc] = z[i] 
        print("information_set McEliece:", information_set_decoding,"\t\t iteration:", self.iteration_dict["McEliece_iteration"] ) 
        return e, information_set_decoding 
    
    def DecodeError_LeeBrickell(self ):
        assert(len( self.ciphertext  ) == self.n)
        z = self.ciphertext.copy()
        sigma = np.arange(self.n)
        found = False
        while True:
            G, z, sigma,information_set_decoding = self.NextConfiguration(z, sigma, "LeeBrickell")
            # check for success
            assert(all(z[:self.k]==0))
            for r in range(self.p + 1):
                for m_support in list(combinations([i for i in range(self.k)], r)):
                    m = np.zeros(self.k, dtype=np.int32)
                    for loc in m_support:
                        m[loc] = 1
                    
                    q = np.mod(z + m.dot(G), self.modulus)
                    
                    assert(sum(q) == r + sum(q[self.k:]))
                    if sum(q[self.k:]) == self.t - r:
                        found = True
                        break
                if found:
                    break
            if found:
                break
        
        # un-permute q
        e = np.zeros(self.n, dtype=np.int32)
        for i, loc in enumerate(sigma):
            e[loc] = q[i]  
        print("information_set brickell:", information_set_decoding,"\t\t iteration:", self.iteration_dict["LeeBrickell_iteration"] )
        return e, information_set_decoding
    
    def DecodeError_Leon(self):
        assert(len(self.ciphertext) == self.n)
        z = self.ciphertext.copy()
        sigma = np.arange(self.n)
        n_loops = 0
        found = False
        while True:
            n_loops += 1
            G, z, sigma, information_set_decoding = self.NextConfiguration( z, sigma,"Leon")
            # check for success
            assert(all(z[: self.k]==0))
            for r in range(self.p + 1):
                for m_support in list(combinations([i for i in range(self.k)], r)):
                    m = np.zeros(self.k, dtype = np.int32)
                    for loc in m_support:
                        m[loc] = 1
                    
                    q = np.mod(z + m.dot(G), self.modulus)
                    # check the first l non-pivot columns first
                    if sum(q[self.k : self.k + self.l]) <= self.p - r:
                        # then check remaining n-k-l columns
                        assert(sum(q) == r + sum(q[self.k : self.k + self.l]) + sum(q[self.k + self.l:]))
                        if sum(q[self.k + self.l:]) == self.t - r - sum(q[self.k : self.k + self.l]):
                            found = True
                            break
                if found:
                    break
            if found:
                break
        
        # un-permute q
        e = np.zeros(self.n, dtype=np.int32)
        print("information_set leon:\t ", information_set_decoding,"\t\t iteration:", self.iteration_dict["Leon_iteration"]  )
        for i, loc in enumerate(sigma):
            e[loc] = q[i]  
        return e, information_set_decoding

    def encodig(self):
        self.Public_key = self.generate_public_key()
        # Choose secret message
        self.original_message = np.random.randint(low = 0, high = 1 + 1 , size = self.k ).astype(np.int32)
        #print("Messagem:\t\t ", self.original_message )
        # Choose encoding error
        error_locations = random.sample([i for i in range(self.n)], self.t)
        Error = np.zeros(shape = self.n, dtype = np.int32)
        for loc in error_locations:
            Error[loc] = 1
    
        print("\n")
    
        # Encode codeword and apply the error
        self.original_codeword   =  np.mod(np.dot(self.original_message,self.Public_key), self.modulus)
        self.ciphertext = np.bitwise_xor(self.original_codeword,Error)
        return self.ciphertext
    
    def main(self):
        self.encodig()
        error_McEliece_information , information_set_decoding  = self.DecodeError_McEliece_information()
        error_LeeBrickell, information_set_decoding_brickell   = self.DecodeError_LeeBrickell( )
        error_Leon, information_set_decoding                   = self.DecodeError_Leon( )
        print("ciphertext\t\t", self.ciphertext)
        print("error using LeeBrickell ", error_LeeBrickell, "\n")
        #self.error_recovery = error_LeeBrickell
        self.error_recovery= error_McEliece_information
         

        self.recover_codeword = np.mod((self.ciphertext - self.error_recovery), self.modulus ) # codeword recuperado 
        if (self.recover_codeword == self.original_codeword  ).all():
            print(" Os codeword sao identicos")     
        else:
            print(" Os codeword não sao identicos") 


       
        G_I = self.Public_key[:,information_set_decoding ] 
        self.recover_codeword_I =  self.recover_codeword [[information_set_decoding]] 
        inverse_matrix = Matrix(G_I).inv_mod( self.modulus)
        M_inv_mod_2_np = np.array(inverse_matrix).astype(int)
    
        result = Matrix(np.mod( np.dot(G_I, M_inv_mod_2_np), self.modulus  ))
       
        recovered_message = (np.mod( np.dot(self.recover_codeword_I,inverse_matrix ), self.modulus ))[0].tolist()
        if (recovered_message == self.original_message).all() :
            print("recovered_message: ",recovered_message )
        else:
            print("Attack failed!!") 



        

         
m, t, p, l = 5, 4, 3,3
attack = Attack(m, t, p, l )
attack.main()



1 trials untils  generated


information_set McEliece: [30, 25, 8, 28, 5, 31, 19, 3, 12, 4, 16, 18] 		 iteration: 25
information_set brickell: [9, 12, 15, 6, 8, 30, 3, 2, 25, 0, 1, 28] 		 iteration: 1
information_set leon:	  [10, 0, 17, 18, 13, 16, 26, 22, 7, 25, 8, 11] 		 iteration: 5
ciphertext		 [0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0]
error using LeeBrickell  [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0] 

 Os codeword não sao identicos
Attack failed!!
